## Running NIDN

We are now ready to get started.
Before we go ahead, we must do the imports.

### Imports 

In [ ]:
### Imports (TODO remove this when finished)
%load_ext autoreload
%autoreload 2

# Append root folder in case you haven't installed NIDN
import sys
sys.path.append("../")

import nidn

In [ ]:
# Load default cfg as starting point
cfg = nidn.load_default_cfg()

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = (1.475)*1e-6
cfg.physical_wavelength_range[1] = (1.525)*1e-6

cfg.N_freq = 32

cfg.target_reflectance_spectrum = [0.99802973,0.99778675,0.99748673,0.99711038,0.99662950,0.99600160,0.99516017,0.99399646,0.99232272,0.98979213,0.98570581,0.97848197,0.96390737,0.92765661,0.79774928,0.13013411,0.59555165,0.88765163,0.95092431,0.97282389,0.98276342,0.98807370,0.99123354,0.99326283,0.99464220,0.99562184,0.99634230,0.99688743,0.99730973,0.99764344,0.99791164,0.99813038,]
cfg.target_transmittance_spectrum = [0.00176019,0.00199939,0.00229478,0.00266539,0.00313898,0.00375744,0.00458628,0.00573266,0.00738156,0.00987468,0.01390061,0.02101778,0.03537726,0.07109298,0.19908235,0.85683378,0.39828372,0.11050055,0.04816339,0.02658768,0.01679511,0.01156331,0.00845010,0.00645072,0.00509164,0.00412636,0.00341642,0.00287921,0.00246302,0.00213410,0.00186971,0.00165405,]

cfg.absorption_loss = True

nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)


# Determine target frequencies (in TRCWA units)
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution
)

physical_wls, normalized_freqs = nidn.get_frequency_points(cfg)
print("Physical wavelengths are (in meters):")
print(physical_wls)

In [ ]:
# Set thicknesses
thicknesses = []
layers = 13
layers_per_part = (layers-1) // 4
n_2 = 2.2
n_1 = 1.5

lam0 = 1.50
d_1 = lam0/(4*n_1)
d_2 = lam0/(4*n_2)
d_defect = lam0/(2*n_1)

# Top DBR
for i in range(layers_per_part):
    thicknesses.append(d_1)
    thicknesses.append(d_2)
    
# Defect 
thicknesses.append(d_defect)

# Bottom DBR
for i in range(layers_per_part,2*layers_per_part):
    thicknesses.append(d_2)
    thicknesses.append(d_1)

thicknesses.reverse()
cfg.TRCWA_PER_LAYER_THICKNESS = thicknesses

## Regression

In [ ]:
cfg.pop("model",None); # Forget the old model
# Set grid specifics
cfg.Nx = 1
cfg.Ny = 1
cfg.N_layers = layers
cfg.TRCWA_L_grid = [[1.,0.0],[0.0,1.]]
cfg.TRCWA_NG = 11

# Allowed range of epsilon values
cfg.real_min_eps = 0.0
cfg.real_max_eps = 10.0
cfg.imag_min_eps = 0.0
cfg.imag_max_eps = 0.1

cfg.type = "regression" # Choose type as described above

In [ ]:
cfg.iterations = 1000
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg,legend=False)

## Classification

In [ ]:
cfg.pop("model",None); # Forget the old model

cfg.type = "classification" # Choose type as described above
cfg.iterations = 1000 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg,legend=False)